In [2]:

# import torch
# print(f"CUDA available: {torch.cuda.is_available()}")
# print(f"Number of GPUs: {torch.cuda.device_count()}")
# print(f"Current GPU: {torch.cuda.current_device()}")
# print(f"GPU Name: {torch.cuda.get_device_name(0)}")
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
torch.cuda.empty_cache()
print(os.getenv("PYTORCH_CUDA_ALLOC_CONF"))


expandable_segments:True


In [3]:
%pwd

'd:\\Text_summarization\\notebook'

In [4]:
os.chdir("../")
%pwd

'd:\\Text_summarization'

In [5]:
# !pip install accelerate
# !pip install transformers accelerate
import accelerate
print(accelerate.__version__)

d:\Text_summarization\myenv\lib\site-packages\tqdm-4.67.1-py3.9.egg\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.6.0


In [6]:
%pwd
# !pip install --upgrade transformers
# !pip uninstall transformers



'd:\\Text_summarization'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_steps: int # added
    evaluation_strategy: str # added
    save_steps: int # changed from float to int
    gradient_accumulation_steps: int

In [8]:

from src.TEXT_SUMMARIZATION.constants import *
from src.TEXT_SUMMARIZATION.common import read_yaml, create_directories

In [9]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig( root_dir=config.root_dir, data_path=config.data_path,
                            model_ckpt=config.model_ckpt,
                            num_train_epochs=params.num_train_epochs, 
                            warmup_steps=params.warmup_steps, 
                            per_device_train_batch_size=params.per_device_train_batch_size,
                            weight_decay=params.weight_decay, 
                            logging_steps=params.logging_steps, 
                            eval_steps=params.eval_steps, 
                            evaluation_strategy=params.evaluation_strategy, 
                            save_steps=params.save_steps, 
                            gradient_accumulation_steps=params.gradient_accumulation_steps)
                         

        return model_trainer_config

In [10]:
# !pip uninstall transformers
# !pip install transformers

from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset,load_from_disk

In [11]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        import gc
        device =  "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments( 
                output_dir=self.config.root_dir, 
                num_train_epochs=self.config.num_train_epochs,
                warmup_steps=self.config.warmup_steps, 
                per_device_train_batch_size=self.config.per_device_train_batch_size,
                weight_decay=self.config.weight_decay, 
                logging_steps=self.config.logging_steps, 
                evaluation_strategy=self.config.evaluation_strategy, 
                eval_steps=self.config.eval_steps, 
                save_steps=self.config.save_steps, 
                gradient_accumulation_steps=self.config.gradient_accumulation_steps,
                fp16 = True)


        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [12]:
from src.TEXT_SUMMARIZATION.exception import CustomException
import sys
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise CustomException(e,sys)

d:\Text_summarization\myenv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Text_summarization\myenv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Text_summarization\myenv\lib\site-packages\torch\nn\modules\module.py:1329: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(
d:\Text_summarization\myenv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be remov

{'loss': 3.2334, 'learning_rate': 9e-07, 'epoch': 0.02}


  3%|▎         | 20/613 [01:29<37:42,  3.82s/it]

{'loss': 3.252, 'learning_rate': 1.9e-06, 'epoch': 0.03}


  5%|▍         | 30/613 [02:16<45:58,  4.73s/it]

...

{'eval_loss': 1.8796818256378174, 'eval_runtime': 59.1282, 'eval_samples_per_second': 13.834, 'eval_steps_per_second': 1.742, 'epoch': 0.98}


100%|█████████▉| 610/613 [47:07<00:16,  5.44s/it]

{'loss': 2.1003, 'learning_rate': 2.6548672566371683e-06, 'epoch': 0.99}


100%|██████████| 613/613 [47:19<00:00,  4.63s/it]


{'train_runtime': 2839.3783, 'train_samples_per_second': 5.188, 'train_steps_per_second': 0.216, 'train_loss': 2.368479538119442, 'epoch': 1.0}


In [12]:
print(torch.__file__)


d:\Text_summarization\myenv\lib\site-packages\torch\__init__.py
